In [1]:
import numpy as np
Id = np.eye(2)
pX = np.array([[0,1.],[1.,0]])
pY = np.array([[0,-1.j],[1.j,0]])
pZ = np.array([[1.,0],[0,-1.]])

In [3]:
L = 2
N_2d = 2*L**2
s = np.arange(N_2d)

### Exact Diagonalization of Toric code model ###
print("### Exact Diagonalization of Toric code model ###")

#construct adjacent lists
plaqlist = [(0,2,3,4),(1,3,5,2),(4,6,7,0),(5,7,1,6)]
vertlist = [(0,2,1,6),(0,3,1,7),(2,4,6,5),(3,5,7,4)]

#define many-body operators
def many_body_operator(idx, oprts, size = N_2d):
    "Tensor product of `oprts` acting on indexes `idx`. Fills rest with Id."
    matrices = [Id if k not in idx else oprts[idx.index(k)] for k in range(size)]
    prod = matrices[0]
    for k in range(1, size):
        prod = np.kron(prod, matrices[k]) 
    return prod

Hamil = np.zeros([2**(N_2d), 2**(N_2d)])+0j

for i in range(L**2):
    Hamil += -1 * many_body_operator(plaqlist[i],[pX,pX,pX,pX])
    Hamil += -1 * many_body_operator(vertlist[i],[pZ,pZ,pZ,pZ])

evals,evecs = np.linalg.eigh(Hamil)
lowest_16_evals = np.sort(evals)[:16]
index = np.argsort(evals)
gsv = evecs[index[0]]
print("The lowest 16 eigenvalues of the 2*2 toric code model is {}".format(lowest_16_evals))

### Construct the 4-fold degenerated ground states using PEPS ###
print("### Construct the 4-fold degenerated ground states using PEPS ###")

v_nontrivial = [(0,0,0,0),(0,1,0,1),(1,0,1,0),(1,0,0,1),(1,1,0,0),(0,1,1,0),(0,0,1,1),(1,1,1,1)]
b_nontrivial = [(0,0,0),(1,1,1)]

vert = np.zeros([2,2,2,2])
bond = np.zeros([2,2,2])

for v_index in v_nontrivial:
    vert[v_index[0], v_index[1], v_index[2], v_index[3]] = 1
for b_index in b_nontrivial:
    bond[b_index[0], b_index[1], b_index[2]] = 1
# contract the tensors to obtain the toric code ground state
peps_tensor = np.einsum("abcd, efgh, ijkl, mnpq, dAj, lBb, cCe, kDm, hEn, qFf, gGa, pHi -> ABCDEFGH",\
                        vert, vert, vert, vert, bond, bond, bond, bond, bond, bond, bond, bond)

peps_vector = peps_tensor.reshape(2**N_2d)

# calculate the energy expectation value of the PEPS tensor and verify that it is indeed the ground state of the toric code model
energy = np.einsum("i,ij,j",peps_vector ,Hamil,peps_vector )/(np.sum(peps_vector**2))
print("Energy of the PEPS state with NO Z-string: {}".format(energy))

# adding closed Z-string around the torus horizontally
peps_tensor1 = np.einsum("abcd, efgh, ijkl, mnpq, dAj, lBb, rCe, sDm, hEn, qFf, gGa, pHi, cr, ks-> ABCDEFGH",\
                        vert, vert, vert, vert, bond, bond, bond, bond, bond, bond, bond, bond, pZ, pZ)

peps_vector1 = peps_tensor1.reshape(2**N_2d)

# calculate the energy expectation value of the PEPS tensor and verify that it is indeed the ground state of the toric code model
energy1 = np.einsum("i,ij,j",peps_vector1 ,Hamil,peps_vector1 )/(np.sum(peps_vector1**2))
print("Energy of the PEPS state with a Z-string AROUND the torus: {}".format(energy1))

# adding closed Z-string across the torus vertically
peps_tensor2 = np.einsum("abcd, efgh, ijkl, mnpq, rAj, lBb, cCe, kDm, sEn, qFf, gGa, pHi, dr, hs -> ABCDEFGH",\
                        vert, vert, vert, vert, bond, bond, bond, bond, bond, bond, bond, bond, pZ, pZ)

peps_vector2 = peps_tensor2.reshape(2**N_2d)

# calculate the energy expectation value of the PEPS tensor and verify that it is indeed the ground state of the toric code model
energy2 = np.einsum("i,ij,j",peps_vector2 ,Hamil,peps_vector2 )/(np.sum(peps_vector2**2))
print("Energy of the PEPS state with a Z-string ACROSS the torus: {}".format(energy2))

# adding closed Z-string both around and across the torus horizontally and vertically
peps_tensor3 = np.einsum("abcd, efgh, ijkl, mnpq, uAj, lBb, rCe, sDm, vEn, qFf, gGa, pHi, cr, ks, du, hv -> ABCDEFGH",\
                        vert, vert, vert, vert, bond, bond, bond, bond, bond, bond, bond, bond, pZ, pZ, pZ, pZ)

peps_vector3 = peps_tensor3.reshape(2**N_2d)

# calculate the energy expectation value of the PEPS tensor and verify that it is indeed the ground state of the toric code model
energy3 = np.einsum("i,ij,j",peps_vector3 ,Hamil,peps_vector3 )/(np.sum(peps_vector3**2))
print("Energy of the PEPS state with Z-strings both AROUND and ACROSS the torus: {}".format(energy3))

### Exact Diagonalization of Toric code model ###
The lowest 16 eigenvalues of the 2*2 toric code model is [-8. -8. -8. -8. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4.]
### Construct the 4-fold degenerated ground states using PEPS ###
Energy of the PEPS state with NO Z-string: (-8+0j)
Energy of the PEPS state with a Z-string AROUND the torus: (-8+0j)
Energy of the PEPS state with a Z-string ACROSS the torus: (-8+0j)
Energy of the PEPS state with Z-strings both AROUND and ACROSS the torus: (-8+0j)
